# Machine Learning

## 1. Import Libs

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn import metrics
from sklearn.feature_selection import f_classif, SelectKBest, RFECV
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

import pandas as pd

## 2. Import datas

In [2]:
train_df = pd.read_csv("train_df.csv")
valid_df = pd.read_csv("valid_df.csv")
test_df = pd.read_csv("test_df.csv")
df = pd.read_csv("../data/radiomics_fundus_ds.csv")

In [3]:
train_df

cl  SID OSOD  calc_energy  calc_entropy  calc_kurtosis  \
0     1  138   OD    716907434      6.333473       0.775021   
1     1   66   OS    548368835      6.009442       1.641892   
2     1  229   OD    690509169      6.285619       1.442059   
3     0  151   OS   1117117944      6.957891       0.470563   
4     1  243   OS   1124929106      7.027759       0.462698   
..   ..  ...  ...          ...           ...            ...   
343   1  227   OD    591668204      6.276923       3.172567   
344   0   42   OS   1381832432      6.734728       1.940732   
345   0   29   OS   1427375743      6.490827       1.294681   
346   1  132   OD   1040404590      6.332057      -0.033956   
347   1   54   OS   1208867473      5.803484       1.396379   

     calc_meanDeviation  calc_skewness  calc_uniformity   calc_mean  ...  \
0             17.197417       0.764147         0.014619   80.766133  ...   
1             13.407227       1.132966         0.019322   70.864854  ...   
2             24.222976       0.307686         0.019182   74.245283  ...   
3             27.850064       0.712426         0.009320   98.447422  ...   
4             31.830187       0.576333         0.009455   96.564297  ...   
..                  ...            ...              ...         ...  ...   
343           19.272594       1.107232         0.017075   70.622129  ...   
344           26.646947      -0.408039         0.012185  110.143906  ...   
345           21.498738       1.341954         0.014177  114.414814  ...   
346           22.318303       0.750949         0.016326   96.961484  ...   
347           12.221348       1.189616         0.022088  107.390342  ...   

         glszm_LAE    glszm_IV    glszm_SZV  glszm_ZP  glszm_LIE   glszm_HIE  \
0     64286.818668  128.482722   535.720342  0.027412   0.044504  184.603491   
1     69233.574713  188.068966   682.039080  0.033984   0.041769  156.991954   
2    133140.677659  159.171476   457.075021  0.023691   0.014242  215.885820   
3      8256.327642  405.518414  1201.270919  0.069209   0.010891  200.590659   
4     70644.624120  122.939261   303.592430  0.022187   0.010958  247.314261   
..             ...         ...          ...       ...        ...         ...   
343   92980.200615  202.814758   488.505765  0.025410   0.014463  170.421214   
344   39624.808603  220.723931   686.438606  0.035869   0.009752  323.737272   
345   29249.958925  158.671398   503.689822  0.032334   0.025064  183.791604   
346   58483.488580  129.567846   310.685625  0.021807   0.005286  379.927452   
347   70864.609665  198.849949   729.857046  0.028896   0.018874  143.844880   

     glszm_LISAE  glszm_HISAE   glszm_LILAE   glszm_HILAE  
0       0.021271    72.806723   3098.782434  4.136360e+06  
1       0.020664    59.567719   2287.973205  5.055437e+06  
2       0.008331    83.829509  30714.345921  1.969283e+07  
3       0.004210    86.061145    289.682098  6.197063e+05  
4       0.005493    78.699063   1389.370614  1.073836e+07  
..           ...          ...           ...           ...  
343     0.008116    67.391669   2493.089589  1.153907e+07  
344     0.005799   139.210104   4685.779659  1.031385e+07  
345     0.010355    68.028708    830.377461  2.250514e+06  
346     0.002532   136.143615    256.083778  1.558137e+07  
347     0.009762    72.149520   1047.701708  8.375632e+06  

[348 rows x 59 columns]

In [4]:
train_x = train_df.drop(columns=["cl", "SID", "OSOD"])
train_y = train_df["cl"]

valid_x = valid_df.drop(columns=["cl", "SID", "OSOD"])
valid_y = valid_df["cl"]

test_x = test_df.drop(columns=["cl", "SID", "OSOD"])
test_y = test_df["cl"]

df_x = df.drop(columns=["cl", "SID", "OSOD"])
df_y = df["cl"]

In [5]:
train_x = pd.concat([train_x, valid_x], axis = 0)
train_y = pd.concat([train_y, valid_y], axis = 0)

## 3. Linear Regression

In [6]:
LR = LinearRegression()
LR_results = cross_validate(LR, df_x, df_y)
print(LR_results['test_score'].mean())
df = pd.DataFrame(LR_results)
df = df.sort_values(by='test_score', ascending=False)
print(df)

-1.0502892988393953
   fit_time  score_time  test_score
3     0.003    0.001999   -0.007013
1     0.003    0.002000   -0.179293
4     0.003    0.002000   -0.212129
2     0.003    0.002000   -2.105672
0     0.012    0.003000   -2.747339


R^2 score가 음수이므로 거의 쓰레기에 가까운 결과이다.

## 4. KNN

In [7]:
KNN = KNeighborsClassifier(n_neighbors=3)
KNN_results = cross_validate(KNN, df_x, df_y)
print(KNN_results['test_score'].mean())
df = pd.DataFrame(KNN_results)
df = df.sort_values(by='test_score', ascending=False)
print(df)

0.5666245259165613
   fit_time  score_time  test_score
4  0.005997    0.005999    0.607143
2  0.005000    0.005000    0.592920
1  0.005000    0.006000    0.584071
3  0.005000    0.006003    0.535714
0  0.004999    0.006000    0.513274


찍는 것에 가까운 수준의 결과이다.

## 5. GridSearch()

In [8]:
rfc = RandomForestClassifier(n_jobs=-1,max_features= 'sqrt' ,n_estimators=50, oob_score = True) 

param_grid = { 
    'n_estimators': [],
    'max_features': ['auto', 'sqrt', 'log2']
}

for i in range(50):
    param_grid['n_estimators'].append((i+1)*10)
    
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(train_x, train_y)
print(CV_rfc.score(train_x,train_y))
print(CV_rfc.score(test_x,test_y))

C:\Users\72200137\.conda\envs\ds\lib\site-packages\sklearn\ensemble\_forest.py:540: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\72200137\.conda\envs\ds\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
C:\Users\72200137\.conda\envs\ds\lib\site-packages\sklearn\ensemble\_forest.py:540: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\72200137\.conda\envs\ds\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
C:\Users\72200137\.conda\envs\ds\lib\site-packages\sklearn\ensemble\_forest.py:540: UserWarning: Some in

1.0
0.650887573964497


random forest를 gridsearch 했을 때 결과는 65.08% 정도로 위의 성능에 비해서 눈에 띄게 나아졌다. 실제 가장 좋은 결과와 비교하면 10퍼센트 정도의 차이가 난다.

## 6. adaboost

In [9]:
clf = AdaBoostClassifier(n_estimators=45, random_state=0)
cv_results = cross_validate(clf, df_x, df_y)
print(cv_results['test_score'].mean())
df = pd.DataFrame(cv_results)
df = df.sort_values(by='test_score', ascending=False)
print(df)

0.5862831858407078
   fit_time  score_time  test_score
4  0.143992    0.009002    0.633929
3  0.144981    0.008002    0.616071
1  0.150007    0.008001    0.610619
2  0.146981    0.009012    0.548673
0  0.206993    0.009003    0.522124


rf보다 더 안좋은 성능이다.

# ANOVA F 검정을 이용한 feature selection 이후

In [10]:
selector1 = SelectKBest(f_classif, k=15)
train_x1 = selector1.fit_transform(train_x, train_y)
test_x1 = selector1.transform(test_x)
df_x1 = selector1.transform(df_x)

In [11]:
cols = selector1.get_support(indices=True)
train_x1 = pd.DataFrame(train_x1, columns = cols)

print(len(selector1.pvalues_))
print(len(selector1.pvalues_[selector1.pvalues_ < 0.05]))
train_x1

56
40


6      7           11        23        24        25        26  \
0     80.766133   79.0   83.672272  0.020215  4.352583  0.717968  0.702472   
1     70.864854   68.0   73.178989  0.025763  4.216713  0.729457  0.716097   
2     74.245283   75.0   82.117316  0.036905  3.906171  0.762673  0.750381   
3     98.447422   96.0  104.447857  0.012596  4.810290  0.615086  0.580986   
4     96.564297   98.0  104.812384  0.020208  4.425026  0.705056  0.685436   
..          ...    ...         ...       ...       ...       ...       ...   
475  115.372080  119.0  119.110391  0.034417  4.028843  0.725545  0.709967   
476   91.643652   89.0  102.952884  0.029011  4.245059  0.710712  0.688506   
477   88.855127   91.0   94.942581  0.052298  3.543549  0.792929  0.784254   
478   98.687031   91.0  104.185270  0.030355  4.175325  0.718961  0.701517   
479   99.024795   98.0  104.381955  0.027898  4.135389  0.730689  0.715020   

           28        29        30          36        41             46  \
0    0.979779  0.429481  0.042306   28.154713  0.275586   64286.818668   
1    0.980639  0.422878  0.076098   24.600575  0.298770   69233.574713   
2    0.982685  0.364301  0.081388  122.457178  0.220869  133140.677659   
3    0.967729  0.456250  0.030587   12.331338  0.388223    8256.327642   
4    0.977856  0.416791  0.063657   34.906504  0.260811   70644.624120   
..        ...       ...       ...         ...       ...            ...   
475  0.979254  0.410655  0.108673   35.800918  0.255273   70171.601813   
476  0.976426  0.373219  0.089695  103.713953  0.276738   53456.838477   
477  0.985124  0.334656  0.097547  147.086950  0.195986  192971.669664   
478  0.979389  0.410029  0.080380   35.757822  0.252510  110060.603633   
479  0.979854  0.397308  0.064221   46.502164  0.243672   56368.660802   

           52            54  
0    0.021271   3098.782434  
1    0.020664   2287.973205  
2    0.008331  30714.345921  
3    0.004210    289.682098  
4    0.005493   1389.370614  
..        ...           ...  
475  0.002486    291.276907  
476  0.005997  24718.570149  
477  0.008568  33045.136851  
478  0.009128   3059.475390  
479  0.006651   3207.439772  

[480 rows x 15 columns]

## 7. Linear Regression

In [12]:
LR = LinearRegression()
LR_results = cross_validate(LR, df_x1, df_y)
print(LR_results['test_score'].mean())
df = pd.DataFrame(LR_results)
df = df.sort_values(by='test_score', ascending=False)
print(df)

-0.1620191982693576
   fit_time  score_time  test_score
4  0.001007    0.000000   -0.036576
1  0.001001    0.000000   -0.051310
3  0.002000    0.001000   -0.187415
2  0.001003    0.000998   -0.204076
0  0.000983    0.001000   -0.330719


확실히 성능이 늘어났다.

## 8. KNN

In [13]:
KNN = KNeighborsClassifier(n_neighbors=15)
KNN_results = cross_validate(KNN, df_x1, df_y)
print(KNN_results['test_score'].mean())
df = pd.DataFrame(KNN_results)
df = df.sort_values(by='test_score', ascending=False)
print(df)

0.6448324905183312
   fit_time  score_time  test_score
3  0.002000    0.004000    0.714286
2  0.002000    0.003000    0.681416
1  0.002000    0.003000    0.619469
4  0.001000    0.003999    0.616071
0  0.000999    0.004999    0.592920


## 9. GridSearch

In [14]:
rfc = RandomForestClassifier(n_jobs=-1,max_features= 'sqrt' ,n_estimators=50, oob_score = True) 

param_grid = { 
    'n_estimators': [],
    'max_features': ['auto', 'sqrt', 'log2']
}

for i in range(50):
    param_grid['n_estimators'].append((i+1)*10)
    
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(train_x1, train_y)
print(CV_rfc.score(train_x1,train_y))
print(CV_rfc.score(test_x1,test_y))

C:\Users\72200137\.conda\envs\ds\lib\site-packages\sklearn\ensemble\_forest.py:540: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\72200137\.conda\envs\ds\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
C:\Users\72200137\.conda\envs\ds\lib\site-packages\sklearn\ensemble\_forest.py:540: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\72200137\.conda\envs\ds\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
C:\Users\72200137\.conda\envs\ds\lib\site-packages\sklearn\ensemble\_forest.py:540: UserWarning: Some in

1.0
0.6035502958579881


## 10. adaboost

In [20]:
clf = AdaBoostClassifier(n_estimators=45, random_state=0)
cv_results = cross_validate(clf, df_x1, df_y)
print(cv_results['test_score'].mean())
df = pd.DataFrame(cv_results)
df = df.sort_values(by='test_score', ascending=False)
print(df)

0.5755214917825537
   fit_time  score_time  test_score
1  0.081997    0.007000    0.601770
4  0.078000    0.005998    0.598214
0  0.131995    0.007001    0.575221
3  0.077012    0.005985    0.571429
2  0.078997    0.005999    0.530973


## 11. adaboost with gridsearch

In [19]:
ada = AdaBoostClassifier(n_estimators=3, random_state=0)

param_grid = { 
    'n_estimators': [],
    'learning_rate': [],
    'algorithm': ['SAMME', 'SAMME.R'],
}

for i in range(27):
    param_grid['n_estimators'].append((i+3))

for i in range(10):
    param_grid['learning_rate'].append(1 - (0.02 * i))
    
ada_rfc = GridSearchCV(estimator=ada, param_grid=param_grid, cv= 5)
ada_rfc.fit(train_x1, train_y)
print(ada_rfc.score(train_x1,train_y))
print(ada_rfc.score(test_x1,test_y))

0.8458333333333333
0.5621301775147929


## 12. Gaussian Naive Bayes

In [22]:
GNB = GaussianNB()
GNB_results = cross_validate(GNB, df_x1, df_y)
print(GNB_results['test_score'].mean())
df = pd.DataFrame(GNB_results)
df = df.sort_values(by='test_score', ascending=False)
print(df)

0.5826011378002527
   fit_time  score_time  test_score
3  0.001000       0.000    0.678571
2  0.001000       0.001    0.646018
1  0.001000       0.000    0.601770
0  0.002000       0.000    0.495575
4  0.000999       0.000    0.491071


## 13. SVC

In [25]:
svc = SVC(C=1) 

param_grid = { 
    'C': [1],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto']
}

for i in range(10):
    param_grid['C'].append((i+1)*5)
    
SVC_rfc = GridSearchCV(estimator=svc, param_grid=param_grid, cv= 5)
CV_rfc.fit(train_x1, train_y)
print(CV_rfc.score(train_x1,train_y))
print(CV_rfc.score(test_x1,test_y))

C:\Users\72200137\.conda\envs\ds\lib\site-packages\sklearn\ensemble\_forest.py:540: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\72200137\.conda\envs\ds\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
C:\Users\72200137\.conda\envs\ds\lib\site-packages\sklearn\ensemble\_forest.py:540: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\72200137\.conda\envs\ds\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
C:\Users\72200137\.conda\envs\ds\lib\site-packages\sklearn\ensemble\_forest.py:540: UserWarning: Some in

1.0
0.5798816568047337


In [ ]:
CV_rfc.fit(train_x, train_y)
print(CV_rfc.score(train_x,train_y))
print(CV_rfc.score(test_x,test_y))

C:\Users\72200137\.conda\envs\ds\lib\site-packages\sklearn\ensemble\_forest.py:540: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\72200137\.conda\envs\ds\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
C:\Users\72200137\.conda\envs\ds\lib\site-packages\sklearn\ensemble\_forest.py:540: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\72200137\.conda\envs\ds\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
C:\Users\72200137\.conda\envs\ds\lib\site-packages\sklearn\ensemble\_forest.py:540: UserWarning: Some in